In [1]:
"""
This notebook contains code for drawing face-detected and random samples from a video, 
annotating on face/no face, and calculating precision, recall, and F-score.
"""

import pandas as pd
import cv2
import os
import ntpath
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import shutil
from collections import defaultdict
from detector import FaceDetector
from ast import literal_eval
!which python

/home/users/agrawalk/miniconda2/envs/headcam/bin/python


In [72]:
def create_sample_json(master_json_path, sample_json_path, sample_size=100):
    sample_df = pd.DataFrame(columns=['vid_name', 'frame', 'face_mtcnn', 'bb_mtcnn'])
    df = pd.read_json(master_json_path)
    dfs = [df[df['vid_name'] == vid] for vid in df['vid_name'].unique()]
    
    sample_df = sample_df.append([df.sample(sample_size) for df in dfs], ignore_index=True)
    sample_df = sample_df.append([df[df['face_mtcnn']].sample(sample_size) for df in dfs], ignore_index=True)
    
    sample_df.to_json(sample_json_path)

In [73]:
def format_num(num):
    """converts int num to string, and adds zeros to get to length 5, if necessary."""
    num = str(num)
    return '0' * (5 - len(num)) + num     

def get_annotation_input():
    """respond 'y' for face, 'n' or '' for no face, 
    'g' to go back and re-annotate prev frame"""
    annot = input('Face? (y/[n]/g[o back]) ').lower()
    while annot not in ['y', 'n', '', 'g']:
        annot = input('Invalid input.\nFace? (y/[n]/g[o back])').lower()
    print('Recorded \'{}\''.format(annot))
    return annot

def annotate_frames(frames_dir, sample_json_path):
    """Interface for quick annotation of face/no face in sample_frame_dir.
    Saves info as 'ground_truth' column in sample frames JSON."""
    df = pd.read_json(sample_json_path) # assumes a header is already present.
    img_paths = [os.path.join(frames_dir, '{0}_frames/image-{1}.jpg'.format(vid_name, format_num(num)))
                 for vid_name, num in zip(df['vid_name'], df['frame'])]
    imgs = [cv2.imread(path) for path in img_paths]
    i = 0
    face_present = np.zeros(df['frame'].shape)
    print('BEGIN ANNOTATING {}'.format(sample_frame_dir)) 
    while i < len(imgs):
        print('\nImage {0}/{1}'.format(i + 1, len(imgs)))
        plt.imshow(imgs[i])
        plt.show()
        annot = get_annotation_input()
        if annot == 'g':
            print('Going back to previous frame')
            i -= 1
        else:
            face_present[i] = (annot == 'y')
            i += 1
    
    df['face_present'] = face_present
    print('Saving annotated df')
    df.to_json(sample_json_path)
    print('Length of annotated df: {}'.format(len(df)))
    print('END ANNOTATING {}'.format(frames_dir))

In [74]:
#run a detector on frames in sample
def run_detector_on_sample(detector_name, frames_dir, sample_json_path):
    df = pd.read_json(sample_json_path) # assumes a header is already present.
    img_paths = [os.path.join(frames_dir, '{0}_frames/image-{1}.jpg'.format(vid_name, format_num(num)))
                 for vid_name, num in zip(df['vid_name'], df['frame'])]
    imgs = [cv2.imread(path) for path in img_paths]
    
    detector = FaceDetector(detector_name)
    detections = [detector.detect_faces(img) for img in imgs]
    face_detected = [len(faces) > 0 for faces in detections]
    df['bb_{}'.format(detector_name)] = detections
    df['face_{}'.format(detector_name)] = face_detected
    df.to_json(sample_json_path)

In [75]:
#calculate/display precision, recall and F-score for each detector
#TODO: visualization
def display_prf(sample_json_path, det_names = ['vj', 'mtcnn']):
    df = pd.read_json(sample_json_path)
    true = df[df['face_present']]
    
    for det_name in det_names:
        pos = df[df['face_{}'.format(det_name)]]
        true_pos = sel[sel['face_present']]
        
        p = len(true_pos) / len(pos)
        r = len(true_pos) / len(true)
        f1 = 2 * p * r / (p + r)
        
        print('DETECTOR: {}'.format(det_name))
        print('positive: {0}, true: {1}, true positive: {2}'.format(len(sel), len(rel), len(true_pos)))
        print('precision: {0}, recall: {1}, F1 Score: {2}\n'.format(p, r, f1))

In [16]:
#pads the number w/ zeros
def openpose_format_num(num):
    num = str(num)
    return '0' * (12 - len(num)) + num    

#apply function on each row, return row with openpose info for keypoint
def create_openpose_col(row, openpose_dir, keypoint):
    fname = '{0}_{1}_keypoints.json'.format(row['vid_name'], 
                                            openpose_format_num(row['frame'] - 1))
    path = os.path.join(openpose_dir, '{}.AVI'.format(row['vid_name']), fname)
    op_df = pd.read_json(path)
    #list of keypoint-lists
    return [person[keypoint] for person in op_df['people'].values] 

#add openpose columns to dataframe
def incorporate_openpose_output(sample_json, openpose_dir):
    df = pd.read_json(sample_json)
    for keypoint in ['pose_keypoints', 'face_keypoints', 'hand_left_keypoints', 'hand_right_keypoints']:
        df[keypoint] = df.apply(lambda row: create_openpose_col(row, openpose_dir, keypoint), axis=1)
    df.to_json(sample_json)

In [17]:
MASTER_PATH = '/scratch/users/agrawalk/headcam-algo/tests/gold_set.json'
OPENPOSE_DIR = '/scratch/users/agrawalk/headcam-algo/tests/openpose_json_output'
incorporate_openpose_output(MASTER_PATH, OPENPOSE_DIR)

/scratch/users/agrawalk/headcam-algo/tests/openpose_json_output/053113-1.AVI/053113-1_000000044346_keypoints.json
   version                                             people
0        1  {'pose_keypoints': [0, 0, 0, 354.771, 181.313,...
/scratch/users/agrawalk/headcam-algo/tests/openpose_json_output/053113-1.AVI/053113-1_000000019093_keypoints.json
Empty DataFrame
Columns: [version, people]
Index: []
/scratch/users/agrawalk/headcam-algo/tests/openpose_json_output/061413-3.AVI/061413-3_000000034786_keypoints.json
/scratch/users/agrawalk/headcam-algo/tests/openpose_json_output/053113-1.AVI/053113-1_000000044346_keypoints.json
   version                                             people
0        1  {'pose_keypoints': [0, 0, 0, 354.771, 181.313,...
/scratch/users/agrawalk/headcam-algo/tests/openpose_json_output/053113-1.AVI/053113-1_000000019093_keypoints.json
Empty DataFrame
Columns: [version, people]
Index: []
/scratch/users/agrawalk/headcam-algo/tests/openpose_json_output/061413-3.AVI

ValueError: ('Expected object or value', 'occurred at index 10')

In [ ]:
SAMPLE_PATH = '/scratch/users/agrawalk/headcam-algo/tests/gold_set_sample.json'
MASTER_PATH = '/scratch/users/agrawalk/headcam-algo/tests/gold_set.json'
create_sample_json(MASTER_PATH, SAMPLE_PATH)

In [8]:
#Testing DataFrame operations
sample_df = pd.DataFrame(columns=['vid_name', 'frame', 'face_mtcnn', 'bb_mtcnn'])
df2 = pd.DataFrame([[1,2,True,np.array([[4,4,4,4]])],
                    [5,6,True,np.array([])]], columns=['vid_name', 'frame', 'face_mtcnn', 'bb_mtcnn'])
sample_df = sample_df.append([df2, df2], ignore_index=True)
sample_df[sample_df['face_mtcnn']]

sample_df.to_json('test.json')
sample_df = pd.read_json('test.json')

sample_df['bb_mtcnn'][0][0]

def test(row):
    return row['vid_name'], row['frame']

sample_df.apply(test, axis=1).values

array([(1, 2), (5, 6), (1, 2), (5, 6)], dtype=object)

In [ ]:
df = pd.read_json('gold_set.json')
# df = df[df['vid_name'] != 'toy']
print(len(df))
# df = df.reset_index(drop=not False)
df['frame'].unique()
df.info()
# df.to_json('gold_set.json')